### Compass Files

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import numpy as np
from evFileProcessing import evFuncs # ev COM functions

In [ ]:
def readCompassPMEL(files, mooring, start):
    dir = []
    temp = []
    time = []
    for file in files:
        try:
            dfCompass=pd.read_csv(file, header = None)
            for dat in dfCompass[1]:
                dir.append(float(dat.split('P')[0][2:]))
                temp.append(float(dat.split('-')[-1][:-3]))
            for t in dfCompass[0]:
                time.append(t)
        except:
            continue
    corrTime = [start+(x- pd.to_datetime(time[5])) for x in  pd.to_datetime(time[5:min(len(time), len(dir))])]
    df = pd.DataFrame({'Datetime':corrTime,'Dir_'+str(mooring):dir[5:min(len(time), len(dir))]})
    df = df.set_index('Datetime')
    return df
    return time, dir

def readCompassAaronia(file, mooring):
    df = pd.read_csv(file)
    df['Datetime'] = pd.to_datetime(df['date'] + ' ' + df['time(local time)'])
    df = df[['Datetime','compass (deg)']]
    df = df.rename(columns={'compass (deg)':'Dir_'+str(mooring)})
    df = df.set_index('Datetime')
    return df

def readCompassOpenTag(files, mooring):
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)
        dfA = pd.concat(dfs)
    dfA = dfA.reset_index()
    return dfA

dfD1Y1C1 = readCompassPMEL(glob('../data/2018/DAFT1/Compass/PMEL/SERIAL/*.txt'), 1, datetime.datetime(2017,8,6,17,0,0))
dfD1Y1C2 = readCompassOpenTag(glob('../data/2018/DAFT1/Compass/OpenTag/*.csv'), 1)
dfD2Y1C1 = readCompassPMEL(glob('../data/2018/DAFT2/Compass/PMEL/SERIAL/*.txt'), 2, datetime.datetime(2017,8,8,6,40,0))
dfD1Y2C1 = readCompassAaronia('../data/2019//DAFT1/Compass/Aaronia/DSFT1Converted_Formatted.csv',1)
dfD2Y2C1 = readCompassPMEL(glob('../data/2019//DAFT2/Compass/PMEL/SERIAL/*.txt'), 2, datetime.datetime(2018,8,12,16,20,0))
dfD3Y2C1 = readCompassPMEL(glob('../data/2019//DAFT3/Compass/PMEL/SERIAL/*.txt'), 3, datetime.datetime(2018,8,14,19,30,0))

ds = [dfD1Y1C1,dfD1Y2C1,dfD2Y1C1,dfD2Y2C1,dfD3Y2C1]
fnames = ['C11_2017-2018_CompassHeading.csv','C11_2018-2019_CompassHeading.csv',
 'C1_2017-2018_CompassHeading.csv','C1_2018-2019_CompassHeading.csv',
 'C4_2018-2019_CompassHeading.csv']
magDec =  [9.6, 10.3, 11.6,12.2,13.4]
ct = 0
for d in ds:
    d = d.rename(columns ={d.columns[0]:'Degrees'})
    d['Magnetic Declination'] = magDec[ct]
    d.to_csv(fnames[ct])
    ct+=1


## Mooring Files

In [ ]:
filesD1Y1 = glob('../data/2018/EVFiles/DAFT1/exports/Loaded*.csv')
filesD1Y2 = glob('../data/2019/EVFiles/DAFT1/exports/Loaded*.csv')
filesD2Y1 = glob('../data/2018/EVFiles/DAFT2/exports/Loaded*.csv')
filesD2Y2 = glob('../data/2019/EVFiles/DAFT2/exports/Loaded*.csv')
filesD3Y2 = glob('../data/2019/EVFiles/DAFT3/exports/Loaded*.csv')

fnames = ['C11_2017-2018_WBAT_70kHz.csv','C11_2018-2019_WBAT_70kHz.csv',
 'C1_2017-2018_WBAT_70kHz.csv','C1_2018-2019_WBAT_70kHz.csv',
 'C4_2018-2019_WBAT_70kHz.csv']
ct = 0
for filel in [filesD1Y1,filesD1Y2,filesD2Y1,filesD2Y2,filesD3Y2]:
    df = evFuncs.evExports.readEvExports(filel)
    df[['layer','sA']].to_csv(fnames[ct])
    ct+=1